# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

In [21]:
genre_counts = df.groupby('genre').count()['message']
genre_counts

genre
direct    10766
news      13054
social     2396
Name: message, dtype: int64

In [29]:
dictionary = {}

for i in df.columns[5:]:
        if i == 'related':
            continue
        value_dict = df[i].value_counts()
        dictionary[i] = value_dict.get(1)
    
print(dictionary)

cat_names = []
cat_values = []
for k ,v in dictionary.items():
    cat_names.append(k)
    cat_values.append(v)

print(cat_names)
print(cat_values)

{'request': 4474, 'offer': 118, 'aid_related': 10860, 'medical_help': 2084, 'medical_products': 1313, 'search_and_rescue': 724, 'security': 471, 'military': 860, 'child_alone': None, 'water': 1672, 'food': 2923, 'shelter': 2314, 'clothing': 405, 'money': 604, 'missing_people': 298, 'refugees': 875, 'death': 1194, 'other_aid': 3446, 'infrastructure_related': 1705, 'transport': 1201, 'buildings': 1333, 'electricity': 532, 'tools': 159, 'hospitals': 283, 'shops': 120, 'aid_centers': 309, 'other_infrastructure': 1151, 'weather_related': 7297, 'floods': 2155, 'storm': 2443, 'fire': 282, 'earthquake': 2455, 'cold': 530, 'other_weather': 1376, 'direct_report': 5075}
['request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', '

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df.columns[5:]

Index(['request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [4]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [7]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [8]:
df.request.value_counts()

0    21742
1     4474
Name: request, dtype: int64

Except for \['id', 'message', 'original', 'genre', 'related'\], all other columns are having 0 or 1. Hence, we can directly use those columns for output dataframe (y). But need to perform some processing for 'genre' and 'related' columns. And then merge previous output dataframe y with the processed values of genre and related.

In [5]:
X = df['message']
y = df[df.columns[5:]]
one_hot_encoded = pd.get_dummies(df[['genre','related']])
y = pd.concat([one_hot_encoded, y], axis=1)

In [17]:
y.columns

Index(['related', 'genre_direct', 'genre_news', 'genre_social', 'request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,"urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
multiOutClF = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', multiOutClF),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)

In [13]:
y_predict.shape

(6554, 39)

In [14]:
y_test.shape

(6554, 39)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def report_results(y_test, y_predict):

    count = 0
    result_grid = pd.DataFrame(columns=['Category', 'Precision', 'Recall', 'f1-score'])

    for cat in y_test.columns:
        #classification_report method returns a dictionary when we set the param output_dict=True
        #but, for some odd reason, it doesn't allow me to do so. (may be this is an old version)
        #hence, I'm getting a string as the result of classification_report function.
        #after doing some string processing, I'm able to retrieve precision, recall and f1-score
        report_str = classification_report(y_test[cat], y_predict[:,count])

        result_grid.set_value(count+1, 'Category', cat)

        category_metrics = report_str.split('\n')[-2].split()

        for (index,j) in enumerate(category_metrics):

            if index == 3:
                result_grid.set_value(count+1, 'Precision', category_metrics[index])
            if index == 4:
                result_grid.set_value(count+1, 'Recall', category_metrics[index])
            if index == 5:
                result_grid.set_value(count+1, 'f1-score', category_metrics[index])
        count += 1

    return result_grid

In [16]:
report_results(y_test, y_predict)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predict

,Category,Precision,Recall,f1-score
1,related,0.76,0.79,0.76
2,genre_direct,0.93,0.93,0.93
3,genre_news,0.94,0.94,0.94
4,genre_social,0.97,0.97,0.97
5,request,0.88,0.88,0.86
6,offer,0.99,0.99,0.99
7,aid_related,0.73,0.73,0.72
8,medical_help,0.90,0.92,0.89
9,medical_products,0.94,0.95,0.93
10,search_and_rescue,0.95,0.97,0.96


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [18]:
parameters = {
    'clf__estimator__min_samples_leaf': [1, 2, 3]
#     'clf__estimator__min_samples_split': [2, 3],
#     'clf__estimator__min_weight_fraction_leaf': [0.0, 0.2, 0.4],
#     'clf__estimator__n_estimators': [10, 15, 20],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred1 = cv.predict(X_test)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [20]:
results1 = report_results(y_test, y_pred1)
results1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predict

,Category,Precision,Recall,f1-score
1,related,0.77,0.79,0.77
2,genre_direct,0.93,0.93,0.93
3,genre_news,0.94,0.94,0.94
4,genre_social,0.98,0.98,0.98
5,request,0.87,0.88,0.86
6,offer,0.99,0.99,0.99
7,aid_related,0.73,0.72,0.71
8,medical_help,0.90,0.92,0.89
9,medical_products,0.94,0.95,0.93
10,search_and_rescue,0.96,0.97,0.96


In [21]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
multiOutClF1 = MultiOutputClassifier(DecisionTreeClassifier())

pipeline1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', multiOutClF1),
])

pipeline1.fit(X_train, y_train)
y_pred2 = pipeline1.predict(X_test)
results2 = report_results(y_test, y_pred2)
results2

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,Category,Precision,Recall,f1-score
1,related,0.73,0.73,0.73
2,genre_direct,0.90,0.89,0.89
3,genre_news,0.91,0.91,0.91
4,genre_social,0.98,0.98,0.98
5,request,0.85,0.85,0.85
6,offer,0.99,0.99,0.99
7,aid_related,0.69,0.70,0.70
8,medical_help,0.90,0.90,0.90
9,medical_products,0.94,0.94,0.94
10,search_and_rescue,0.95,0.96,0.95


### 9. Export your model as a pickle file

In [23]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.